In [2]:
!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
!pip install rdkit-pypi

  Using cached torch_geometric-2.5.3-py3-none-any.whl.metadata (64 kB)
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_spline_conv-1.2.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
Using cached torch_geometric-2.5.3-py3-none-any.whl (1.1 MB)
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=507267 sha256=be3b938fe85308d36d1907409e46e9d3a89290faa11a259c4f88e7f74a049f49
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1092566 sha256=cd4cd85449add2344d222d8cc3842ddf50b41099b064ff00cace40cd15af4005
  Stored in directory: /root/.cache/p

In [3]:
import pandas as pd
from google.colab import files

# Upload the dataset
uploaded = files.upload()

# Load the dataset
df = pd.read_csv('acute tox inp.csv')  # Replace with your filename

# Display the first few rows to inspect
df.head()


Saving acute tox inp.csv to acute tox inp.csv


,TAID,Name,IUPAC Name,PubChem CID,Canonical SMILES,InChIKey,Toxicity Value
0,TOX-36930,NaN,9-[4-(4-fluorophenyl)-4-oxobutyl]-4-phenyl-1-o...,108300,O=C1NC(c2ccccc2)CC2(CCN(CCCC(=O)c3ccc(F)cc3)CC...,PXVDNKCZHCXDAP-UHFFFAOYSA-N,500.013399
1,TOX-78100,NaN,N-(cyclohexylideneamino)-1-pyridin-3-ylmethana...,21146036,c1cncc(CNN=C2CCCCC2)c1,QZUAYMFATFVGOI-UHFFFAOYSA-N,249.988932
2,TOX-46841,NaN,2-(dimethylamino)-10-methylbenzo[g]chromen-4-one,3052513,Cc1c2ccccc2cc2c(=O)cc(N(C)C)oc12,CDOTXBDXUGPLCZ-UHFFFAOYSA-N,799.965259
3,TOX-76640,NaN,1-methyl-2-nitro-5-(2-nitrohex-1-enyl)imidazole,105407,CCCCC(=Cc1cnc([N+](=O)[O-])n1C)[N+](=O)[O-],VNDAPXAHSWGLCJ-UHFFFAOYSA-N,499.988200
4,TOX-30322,NaN,"N-(3,5-ditert-butyl-4-hydroxyphenyl)prop-2-ena...",215005,C=CC(=O)Nc1cc(C(C)(C)C)c(O)c(C(C)(C)C)c1,HMUWRTAPFAQCGK-UHFFFAOYSA-N,1000.029050


In [5]:
from rdkit import Chem
from torch_geometric.data import Data
import torch

def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Invalid SMILES string: {smiles}")

    atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    x = torch.tensor(atoms, dtype=torch.float).view(-1, 1)

    edge_index = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append((i, j))
        edge_index.append((j, i))

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    data = Data(x=x, edge_index=edge_index)

    return data

df['graph'] = df['Canonical SMILES'].apply(smiles_to_graph)


In [6]:
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import Transformer

class GNNTransformerModel(nn.Module):
    def __init__(self):
        super(GNNTransformerModel, self).__init__()
        self.gcn1 = GCNConv(1, 64)
        self.gcn2 = GCNConv(64, 128)
        self.pool = global_mean_pool
        self.transformer = Transformer(d_model=128, nhead=8, num_encoder_layers=2)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.gcn1(x, edge_index)
        x = self.gcn2(x, edge_index)
        x = self.pool(x, batch)
        x = x.unsqueeze(1)
        x = self.transformer(x, x)
        x = x.mean(dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


In [10]:
from torch_geometric.data import DataLoader


In [11]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import Transformer
from rdkit import Chem
import pandas as pd
from sklearn.model_selection import LeaveOneOut
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
from torch_geometric.loader import DataLoader  # Corrected import

# Updated model code
class GNNTransformerModel(nn.Module):
    def __init__(self):
        super(GNNTransformerModel, self).__init__()
        self.gcn1 = GCNConv(1, 64)
        self.gcn2 = GCNConv(64, 128)
        self.pool = global_mean_pool
        self.transformer = Transformer(d_model=128, nhead=8, num_encoder_layers=2, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.gcn1(x, edge_index)
        x = self.gcn2(x, edge_index)
        x = self.pool(x, batch)
        x = x.unsqueeze(1)  # Add sequence dimension for Transformer
        x = self.transformer(x, x)
        x = x.mean(dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Implement LOOCV
loo = LeaveOneOut()

mse_scores = []
rmse_scores = []
mae_scores = []
r2_scores = []

for i, (train_index, test_index) in enumerate(loo.split(df)):
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]

    train_data = [smiles_to_graph(smiles) for smiles in train_df['Canonical SMILES']]
    test_data = [smiles_to_graph(smiles) for smiles in test_df['Canonical SMILES']]

    for j, data in enumerate(train_data):
        data.y = torch.tensor([train_df.iloc[j]['Toxicity Value']], dtype=torch.float)
    for j, data in enumerate(test_data):
        data.y = torch.tensor([test_df.iloc[j]['Toxicity Value']], dtype=torch.float)

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

    model = GNNTransformerModel()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    model.train()
    for epoch in range(10):
        for batch in train_loader:
            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, batch.y.view(-1, 1))  # Ensure target reshaping
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for batch in test_loader:
            outputs = model(batch)
            y_true.extend(batch.y.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred) if len(y_true) > 1 else float('nan')

    mse_scores.append(mse)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    r2_scores.append(r2)

    # Save the model for this fold
    torch.save(model.state_dict(), f'gnn_transformer_model_fold_{i}.pth')

    # Print fold results
    print(f"Fold {i+1} - MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R²: {r2}")

# Calculate and print average metrics across all folds
average_mse = np.mean(mse_scores)
average_rmse = np.mean(rmse_scores)
average_mae = np.mean(mae_scores)
average_r2 = np.nanmean(r2_scores)  # Use nanmean to skip NaN values

print(f"\nAverage MSE: {average_mse}")
print(f"Average RMSE: {average_rmse}")
print(f"Average MAE: {average_mae}")
print(f"Average R² Score: {average_r2}")


Fold 1 - MSE: 1255323.875, RMSE: 1120.412353515625, MAE: 1120.412353515625, R²: nan
Fold 2 - MSE: 2119231.75, RMSE: 1455.7581787109375, MAE: 1455.7581787109375, R²: nan
Fold 3 - MSE: 928241.125, RMSE: 963.4526977539062, MAE: 963.4526977539062, R²: nan
Fold 4 - MSE: 1471223.625, RMSE: 1212.9400634765625, MAE: 1212.9400634765625, R²: nan
Fold 5 - MSE: 524567.0625, RMSE: 724.27001953125, MAE: 724.27001953125, R²: nan
Fold 6 - MSE: 20000.4921875, RMSE: 141.423095703125, MAE: 141.423095703125, R²: nan
Fold 7 - MSE: 2730914.5, RMSE: 1652.5478515625, MAE: 1652.5478515625, R²: nan
Fold 8 - MSE: 1171950.5, RMSE: 1082.566650390625, MAE: 1082.566650390625, R²: nan
Fold 9 - MSE: 1541942.5, RMSE: 1241.749755859375, MAE: 1241.749755859375, R²: nan
Fold 10 - MSE: 464478.9375, RMSE: 681.5269165039062, MAE: 681.5269165039062, R²: nan
Fold 11 - MSE: 17322.18359375, RMSE: 131.61376953125, MAE: 131.61376953125, R²: nan
Fold 12 - MSE: 22246.09375, RMSE: 149.1512451171875, MAE: 149.1512451171875, R²: nan
Fo

In [3]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Ensure the DataFrame df is loaded
# If you've already uploaded the file, you can do:
uploaded = files.upload()
df = pd.read_csv('your_file.csv')  # Replace 'your_file.csv' with the actual filename

# Now, you can proceed with the LOOCV loop
loo = LeaveOneOut()

mse_scores = []
rmse_scores = []
mae_scores = []
r2_scores = []

# Adjust the code to simply skip R² calculation
for i, (train_index, test_index) in enumerate(loo.split(df)):
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]

    train_data = [smiles_to_graph(smiles) for smiles in train_df['smiles']]
    test_data = [smiles_to_graph(smiles) for smiles in test_df['smiles']]

    for j, data in enumerate(train_data):
        data.y = torch.tensor([train_df.iloc[j]['Toxicity Value']], dtype=torch.float)
    for j, data in enumerate(test_data):
        data.y = torch.tensor([test_df.iloc[j]['Toxicity Value']], dtype=torch.float)

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

    model = GNNTransformerModel()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    model.train()
    for epoch in range(10):
        for batch in train_loader:
            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, batch.y.view(-1, 1))  # Ensure target reshaping
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for batch in test_loader:
            outputs = model(batch)
            y_true.extend(batch.y.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = None  # Skip R² calculation

    mse_scores.append(mse)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    r2_scores.append(r2)

    # Save the model for this fold
    torch.save(model.state_dict(), f'gnn_transformer_model_fold_{i}.pth')

    # Print fold results
    print(f"Fold {i+1} - MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R²: {r2}")

# Calculate and print average metrics across all folds
average_mse = np.mean(mse_scores)
average_rmse = np.mean(rmse_scores)
average_mae = np.mean(mae_scores)

print(f"\nAverage MSE: {average_mse}")
print(f"Average RMSE: {average_rmse}")
print(f"Average MAE: {average_mae}")


NameError: name 'files' is not defined